# NPS calculated for multiple form fields through time as a Time Series

In [ ]:
import pandas as pd
from configobj import ConfigObj    # dnf install python3-configobj
import sqlalchemy
import matplotlib.pyplot as plt
import numpy as np
import time
import datetime
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
#plt.rcParams['figure.dpi']=400

config='../syncFromTypeform.conf'

In [ ]:
context=ConfigObj(config)
db=sqlalchemy.create_engine(context['database'], encoding='utf8')

## Forms and fields that can be found in our database

In [ ]:
myNPSfields=[
#     ['APiACy', '4772be8bd4f21ea2'],
#     ['KPbhd6', '43afcbf9b69c2feb'],
#     ['kVGmcU', '43afcbf9b69c2feb'],
    ['YRyBYh', '4629726316f59ad4' , 'Renovação de seguros'],
    ['Tz3iJK','353eb07c-15bf-4669-9793-9c0ec33f818a', 'Cotação'],
    ['Eh4x2E','268ef6ae60b480fc', 'Vibra'],
    ['ARqhAx','de80f7ff-2a0b-46f1-afec-e61432da10bf','NPS página inicial de 05/2020, seguradora'],
    ['ARqhAx','33147541-e204-41ad-83ca-777f3f60290a','NPS página inicial de 05/2020, portal'],
#     ['fHUNzp', 'e6dc74518142df7a'],
#     ['WwuFx0', '4a2c5e69-166e-4e44-9543-a0966b8d35f0'],
#     ['WwuFx0', 'f5c7ac13-868f-4948-9195-c812ad851aa3'],
    ['qneoZH','353eb07c-15bf-4669-9793-9c0ec33f818a', 'Cotação (nova)'],
    ['to6Bfp', 'nps', 'Gestão de Negócios'],
    ['WwuFx0', 'nps', 'Tela Inicial (antigo)'],
    ['RwBmJ4','268ef6ae60b480fc', 'Vibra - acompanhamento de inspeção'],
    ['zdgJ8H','59759776-9cb2-4ee0-bac8-1893ae1474af'],
    ['PBBWaL','nps', 'Vila Mobile - Quero Opinar']
]

In [ ]:
fields=pd.DataFrame(myNPSfields,columns=['form_id','field_name', 'comment'])

## NPS evolution calculated and graphed based on answers from all the fields above

In [ ]:
nps=pd.DataFrame()
for index, field in fields.iterrows():
    r=pd.read_sql_query("SELECT * FROM tf_nps_daily WHERE form_id='{}' AND field_name='{}'".format(field['form_id'],field['field_name']),db)
    nps=nps.append(r)
del r

In [ ]:
nps['date']=pd.to_datetime(nps['date'],infer_datetime_format=True)
nps['NPS_cumulative_percent']=100*nps['NPS_cumulative']

In [ ]:
for index, field in fields.iterrows():
    if nps[(nps['form_id']==field['form_id']) & (nps['field_name']==field['field_name'])].shape[0]>0:
        ax=nps[(nps['form_id']==field['form_id']) & (nps['field_name']==field['field_name'])].sort_values(by='date').plot(
            kind='line',
            x='date',
            y='totl_cumulative',
        )


        nps[(nps['form_id']==field['form_id']) & (nps['field_name']==field['field_name'])].sort_values(by='date').plot(
            kind='line',
            x='date',
            y='NPS_cumulative_percent',
            secondary_y='NPS_cumulative_percent',
            ax=ax
        )

        ax.set_title('«{}» {}'.format(
            nps[(nps['form_id']==field['form_id']) & (nps['field_name']==field['field_name'])]['form_id'][0],
            nps[(nps['form_id']==field['form_id']) & (nps['field_name']==field['field_name'])]['form_title'][0],

            nps[(nps['form_id']==field['form_id']) & (nps['field_name']==field['field_name'])]['field_name'][0],
            nps[(nps['form_id']==field['form_id']) & (nps['field_name']==field['field_name'])]['field_title'][0]
        ))

        ax.legend(['# respostas', '% NPS'], loc='best',bbox_to_anchor=(1.1,1))
        ax.set_xlabel('Data')
        ax.set_ylabel('# respostas')
        ax.right_ax.set_ylabel('% NPS')

# Response count per week

In [ ]:
answers=pd.DataFrame()
for index, field in fields.iterrows():
    r=pd.read_sql_query("SELECT * FROM tf_super_answers WHERE form_id='{}' AND field_name='{}'".format(field['form_id'],field['field_name']),db)
    answers=answers.append(r)
del r

In [ ]:
ans=answers[['form_id','submitted','answer']]
ans['answer']=pd.to_numeric(ans['answer'])
ans.set_index('submitted',inplace=True)

# Plain textual data

In [ ]:
nps

In [ ]:
nps=pd.DataFrame()
for index, field in fields.iterrows():
    r=pd.read_sql_query("SELECT * FROM tf_nps WHERE form_id='{}' AND field_name='{}'".format(field['form_id'],field['field_name']),db)
    nps=nps.append(r)
del r

In [ ]:
nps

## Form conversion, or, how many printed forms are actually submmited by users

In [ ]:
query="""
                        SELECT
                            tot.form,
                            tf_forms.title,
                            tot.date,
                            sub.avg_time,
                            tot.landings,
                            sub.submissions,
                            sub.submissions / tot.landings as rate
                        from
                            (
                            select
                                form,
                                cast(landed as date) date,
                                landed,
                                submitted,
                                count(*) landings
                            from
                                tf_responses
                            group by
                                form, date ) as tot
                        left outer join (
                            select
                                form,
                                cast(landed as date) date,
                                avg(time_to_sec(timediff(submitted,landed))) as avg_time,
                                count(*) submissions
                            from
                                tf_responses
                            where
                                submitted is not null
                            group by
                                form, date ) as sub on
                            sub.form = tot.form and sub.date=tot.date
                        left outer join tf_forms
                        on tf_forms.id=tot.form
"""

conversion=pd.DataFrame()
for index, field in fields.iterrows():
    r=pd.read_sql_query(query + " WHERE tot.form='{}'".format(field['form_id']),db)
    conversion=conversion.append(r)
del r

In [ ]:
conversion

In [ ]:
conversion['date']=pd.to_datetime(conversion['date'],infer_datetime_format=True)
conversion['abandoned']=conversion['landings']-conversion['submissions']
conversion.fillna(0, inplace=True)

In [ ]:
conversion.info()

In [ ]:
conversion[conversion['form']=='ARqhAx'] #['submissions'].sum()

In [ ]:
mean=conversion[conversion['form']=='ARqhAx']['avg_time'].mean()
median=conversion[conversion['form']=='ARqhAx']['avg_time'].median()
stddev=conversion[conversion['form']=='ARqhAx']['avg_time'].std()

deltamean=datetime.timedelta(seconds=mean)
print("Mean ({}s): {}m{}s".format(deltamean.seconds, deltamean.seconds//60, deltamean.seconds-(deltamean.seconds//60)))

deltamedian=datetime.timedelta(seconds=median)
print("Median ({}s): {}m{}s".format(deltamedian.seconds, deltamedian.seconds//60, deltamedian.seconds-(deltamedian.seconds//60)))

print("Std deviation ({}s): {}m{}s".format(stddev, stddev//60, stddev-(stddev//60)))



# conversion[conversion['form']=='ARqhAx'].describe()

In [ ]:
for index, field in fields.iterrows():
    if conversion[(conversion['form']==field['form_id'])].shape[0]>0:
        
        conversion_by_week=conversion[(conversion['form']==field['form_id'])][['date','abandoned','submissions','avg_time']].resample(rule='W-MON', on='date').agg({
                    'abandoned': np.sum,
                    'submissions': np.sum,
                    'avg_time': np.mean
                })
        
        conversion_by_week['time_avg_time'] = pd.to_datetime(conversion_by_week['avg_time'], unit='s')        
        conversion_by_week['avg_time_text'] = conversion_by_week['time_avg_time'].dt.strftime('%mm%ss')
#         conversion_by_week[conversion_by_week['avg_time']<60]['avg_time_text'] = conversion_by_week['avg_time'].dt.strftime('%ss')        
        
        ax=conversion_by_week[['submissions','abandoned']].plot(
            kind='bar',
            figsize=(20,8),
            stacked=True, width=0.95
        )

        text = "{} is average time to answer 1 form\n{:.2f}% of form response probability".format(
                    time.strftime("%Mm%Ss",time.gmtime(conversion[(conversion['form']==field['form_id'])]['avg_time'].mean())),
                    conversion[(conversion['form']==field['form_id'])]['submissions'].sum()/conversion[(conversion['form']==field['form_id'])]['landings'].sum()*100
        )
        
        box = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

        ax.text(0.005,0.98, text, transform=ax.transAxes,
                verticalalignment='top', bbox=box
        )
        
#         conversion_by_week[['avg_time']].plot(
#             kind='line',
#             secondary_y='avg_time',
#            # fmt='bo-',
#             ax=ax
#         )

#         for d,avg in conversion_by_week[['avg_time']].iterrows():
#             if np.isnan(avg[0]):
#                 pass
#             else:
#                 label = time.strftime("%M:%S",time.gmtime(avg[0]))

#                 ax.annotate(label, # this is the text
#                              (d,avg[0]), # this is the point to label
#                              textcoords="offset points", # how to position the text
#                              xytext=(0,10), # distance from text to points (x,y)
#                              ha='center',  # horizontal alignment can be left, right or center
#                             ) # horizontal alignment can be left, right or center

        
        
        
        (form,title)=conversion[(conversion['form']==field['form_id'])][['form','title']].loc[0]
                
        ax.set_title('«{}» {}'.format(
            form,
            title
        ))

#         ax.legend(['# respostas', '% NPS'], loc='best',bbox_to_anchor=(1.1,1))
        ax.set_xlabel('Data')
        ax.set_ylabel('# respostas')
#         ax.right_ax.set_ylabel('tempo médio para responder')